# LangChain Hugging Face Agent Mini-Project

This notebook walks through building a mini-project that showcases how to combine [LangChain](https://python.langchain.com/) with a Hugging Face large language model (LLM) to create a simple AI agent. The agent can choose between multiple tools to solve tasks, which makes it an engaging classroom demonstration of *agentic* behavior.

The workflow below is Colab-friendly—you can open this notebook in Google Colab and run it top-to-bottom. Each step is intentionally documented so you can highlight the agent's reasoning process when presenting it live.

## 1. Install dependencies

Google Colab already ships with many scientific Python packages, but we still need to install LangChain and the Hugging Face Transformers stack. The `--quiet` flag keeps the output readable during demos.

In [ ]:
import os
import subprocess
import sys

packages = [
    'langchain==0.2.7',
    'langchain-community==0.2.7',
    'transformers==4.39.3',
    'accelerate==0.28.0',
    'sentencepiece==0.1.99',
]

if os.environ.get('SKIP_COLAB_INSTALL', '').lower() in {'1', 'true', 'yes'}:
    print('Skipping pip install because SKIP_COLAB_INSTALL is set.')
else:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--quiet', *packages])


## 2. Import libraries and configure the runtime

We load the required libraries and make sure LangChain prints rich traces. The helper function below performs safe mathematical evaluations so the agent can handle numeric reasoning tasks without executing arbitrary Python code.

In [ ]:
import math
import os
import re
from typing import Dict

from langchain.agents import AgentType, initialize_agent
from langchain.tools import Tool

try:
    from langchain_community.llms import HuggingFacePipeline
except ImportError:
    try:
        from langchain.llms import HuggingFacePipeline  # Fallback for older LangChain versions
    except ImportError:
        HuggingFacePipeline = None

# Silence overly verbose transformers logging to keep demo output readable.
import logging
logging.getLogger('transformers').setLevel(logging.ERROR)

USE_FAKE_LLM = os.environ.get('USE_FAKE_LLM', '').lower() in {'1', 'true', 'yes'}


def safe_eval(expression: str) -> float:
    """Safely evaluate a basic math expression consisting of numbers and + - * / ^ parentheses."""
    allowed_chars = set('0123456789+-*/(). ^')
    if not set(expression).issubset(allowed_chars):
        raise ValueError('Expression contains unsupported characters.')
    expression = expression.replace('^', '**')
    return eval(expression, {'__builtins__': {}}, {'math': math})


## 3. Load a Hugging Face LLM

We use the open-source **google/flan-t5-base** model, which runs comfortably on the free Colab T4 GPU (or even CPU). Wrapping the Transformers pipeline with LangChain's `HuggingFacePipeline` class gives us a drop-in LLM object that the agent can call.

In [ ]:
if USE_FAKE_LLM:
    print('Using offline demo mode — Hugging Face model will not be downloaded.')
    llm = None
else:
    if HuggingFacePipeline is None:
        raise ImportError('HuggingFacePipeline is required when USE_FAKE_LLM is not set.')
    from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

    model_name = 'google/flan-t5-base'

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    text2text_pipeline = pipeline(
        'text2text-generation',
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=256,
    )
    llm = HuggingFacePipeline(pipeline=text2text_pipeline)


## 4. Build domain-specific tools

To demonstrate agentic behavior, we define three custom tools. Each tool solves a different class of task:

1. **Calculator** – performs arithmetic with `safe_eval`.
2. **Unit Converter** – supports a few classroom-friendly conversions.
3. **Note Summarizer** – uses the LLM directly to condense longer text snippets.

Because the tools return plain strings, they're easy to inspect while explaining how the agent reasons.

In [ ]:
CONVERSION_FACTORS: Dict[str, float] = {
    'kilometers_to_miles': 0.621371,
    'miles_to_kilometers': 1.60934,
    'celsius_to_fahrenheit': None,
}


def convert_units(query: str) -> str:
    query = query.lower().strip()
    match = re.search(r'-?\d+(?:\.\d+)?', query)
    if not match:
        raise ValueError('Please include a number in your conversion request.')
    number = float(match.group())
    if 'kilometer' in query and 'mile' in query:
        miles = number * CONVERSION_FACTORS['kilometers_to_miles']
        return f"{number} kilometers is approximately {miles:.2f} miles."
    if 'mile' in query and 'kilometer' in query:
        kilometers = number * CONVERSION_FACTORS['miles_to_kilometers']
        return f"{number} miles is approximately {kilometers:.2f} kilometers."
    if 'celsius' in query and 'fahrenheit' in query:
        fahrenheit = (number * 9 / 5) + 32
        return f"{number}°C is {fahrenheit:.1f}°F."
    raise ValueError('Unsupported conversion request. Try kilometers↔miles or Celsius↔Fahrenheit.')


def summarize_notes(text: str) -> str:
    if USE_FAKE_LLM:
        sentences = [sentence.strip() for sentence in text.split('.') if sentence.strip()]
        if not sentences:
            return 'Summary: No content provided.'
        summary = '; '.join(sentences[:2])
        return f'Summary: {summary}.'
    prompt = f"Summarize the following classroom note in 2 concise sentences:\n\n{text}\n\nSummary:"
    result = llm(prompt)
    return result.strip()


def evaluate_math(expression: str) -> str:
    value = safe_eval(expression)
    return f'The result is {value}.'


tools = [
    Tool(
        name='Calculator',
        func=evaluate_math,
        description="Useful for solving arithmetic expressions. Input should be a math expression like '23 * (4 + 2)'.",
    ),
    Tool(
        name='Unit Converter',
        func=convert_units,
        description='Converts simple measurements such as kilometers to miles, miles to kilometers, or Celsius to Fahrenheit.',
    ),
    Tool(
        name='Note Summarizer',
        func=summarize_notes,
        description='Summarizes provided lecture notes into a concise overview.',
    ),
]


## 5. Assemble the agent

We create a Zero-Shot ReAct agent, which means it will reason about which tool to call based on the tool descriptions. Setting `verbose=True` ensures the agent prints its intermediate reasoning steps—perfect for classroom walkthroughs.

In [ ]:
if USE_FAKE_LLM:
    agent = None
    print('Skipping LangChain agent initialization in offline mode. Tool functions remain available for manual demos.')
else:
    agent = initialize_agent(
        tools=tools,
        llm=llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
    )


## 6. Demonstrate three agentic tasks

Each of the following prompts highlights a different capability. Run the cell live to show students how the agent selects tools, executes them, and returns natural-language answers.

In [ ]:
tasks = [
    'What is (23 * 4) + 19?',
    'Convert 5 kilometers to miles.',
    'Summarize this: Photosynthesis allows plants to convert light energy into chemical energy. Chlorophyll captures sunlight, water from roots supplies electrons, and carbon dioxide from the air forms glucose. Oxygen is released as a byproduct.',
]

if USE_FAKE_LLM:
    print('Offline demo mode: invoking tools directly to showcase expected outputs.')
    print('\n=== Calculator Task ===')
    print(tasks[0])
    print(evaluate_math('(23 * 4) + 19'))

    print('\n=== Unit Converter Task ===')
    print(tasks[1])
    print(convert_units('5 kilometers to miles'))

    print('\n=== Summarization Task ===')
    print(tasks[2])
    note_text = tasks[2].split('Summarize this: ', 1)[1]
    print(summarize_notes(note_text))
else:
    for task in tasks:
        print('\n=== New Task ===')
        print(task)
        result = agent.run(task)
        print('Answer:', result)


## 7. Next steps for the classroom

* Ask learners to add new tools (e.g., a date planner or trivia lookup) and observe how the agent adapts.
* Swap `google/flan-t5-base` for a more capable Hugging Face model if you have GPU resources.
* Encourage experimentation with different agent types such as the Conversational ReAct agent.

Happy teaching!